In [39]:
from get_info import extract_info
import csv
import math
import ast
import time
import sys

In [2]:
"""
trending1.csv links
    "https://github.com/PaddlePaddle/PaddleSpeech", 
    "https://github.com/terra-money/core",
    "https://github.com/payloadcms/payload",
    "https://github.com/donnemartin/system-design-primer",
    "https://github.com/facebook/flipper",
    "https://github.com/InoriRus/Kyty",
    "https://github.com/ml-tooling/best-of-ml-python",
    "https://github.com/guchangan1/All-Defense-Tool",
    "https://github.com/dair-ai/Mathematics-for-ML",
    "https://github.com/dotnet/aspnetcore",
"""

"""
trending.csv links
    "https://github.com/rocboss/paopao-ce",
    "https://github.com/hasinhayder/hydra",
    "https://github.com/coolsnowwolf/lede",
    "https://github.com/livebud/bud",
    "https://github.com/github/copilot-docs", #5
    "https://github.com/trekhleb/javascript-algorithms",
    "https://github.com/Grasscutters/Grasscutter",
    "https://github.com/payloadcms/payload",
    "https://github.com/cloudflare/cloudflare-docs",
    "https://github.com/XiaoMiku01/fansMedalHelper", #10
    "https://github.com/godotengine/godot", 
    "https://github.com/smartcontractkit/full-blockchain-solidity-course-js", 
    "https://github.com/bregman-arie/devops-exercises", 
    "https://github.com/zijie0/HumanSystemOptimization",
    "https://github.com/Asabeneh/30-Days-Of-React", #15
    "https://github.com/helix-editor/helix", 
    "https://github.com/openwrt/openwrt",
    "https://github.com/trustwallet/assets",
    "https://github.com/qmk/qmk_firmware",
    "https://github.com/lucidrains/imagen-pytorch", #20
"""

'\ntrending1.csv links\n    "https://github.com/PaddlePaddle/PaddleSpeech", \n    "https://github.com/terra-money/core",\n    "https://github.com/payloadcms/payload",\n    "https://github.com/donnemartin/system-design-primer",\n    "https://github.com/facebook/flipper",\n    "https://github.com/InoriRus/Kyty",\n    "https://github.com/ml-tooling/best-of-ml-python",\n    "https://github.com/guchangan1/All-Defense-Tool",\n    "https://github.com/dair-ai/Mathematics-for-ML",\n    "https://github.com/dotnet/aspnetcore",\n'

In [58]:
# top 10 trending links as of 05/29/2022
trending_links = [
    "https://github.com/PaddlePaddle/PaddleSpeech", 
    "https://github.com/terra-money/core",
    "https://github.com/payloadcms/payload",
    "https://github.com/donnemartin/system-design-primer",
    "https://github.com/facebook/flipper",
    "https://github.com/InoriRus/Kyty",
    "https://github.com/ml-tooling/best-of-ml-python",
    "https://github.com/guchangan1/All-Defense-Tool",
    "https://github.com/dair-ai/Mathematics-for-ML",
    "https://github.com/dotnet/aspnetcore",
]

data = []
for i in range(len(trending_links)):
    print(i)
    data.append(extract_info(trending_links[i]))

0
1
2
3
4
5
6
7
8
9


In [59]:
with open("trending.csv", "w") as file: # to append, change "w" to "a"
    fieldnames = ['num_stars', 'num_forks', 'num_open_issues', 'num_contributors', 'num_closed_issues']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader() # to append, comment out line
    for i in range(len(data)):
        writer.writerow(data[i])

In [62]:
trending_data = []
with open("trending.csv", "r") as file: 
    reader = csv.DictReader(file)
    for row in reader:
        trending_data.append(row)

In [7]:
# Spearman Correlation Coefficient calculation
# https://www.statisticshowto.com/probability-and-statistics/correlation-coefficient-formula/spearman-rank-correlation-definition-calculate/
def rank_correlation(arr):
	ans = 0
	temp = arr.copy()
	h = {}
	n = len(arr)
	temp.sort(reverse=True)

	for i in range(n):
		h[arr[i]] = i

	for i in range(n):
		if (arr[i] != temp[i]):
			ans += (abs(i - h[temp[i]]))**2
	
	return (1 - 6*ans/(n*(n**2-1)))

In [63]:
epochs = [1653656820, 1653730620, 1653500640, 1650719640, 1653738420,
          1653698280, 1653584520, 1653733080, 1653418680, 1653788880]

epochs = [1653813000, 1653850560, 1653840300, 1653838800, 1650964200,
          1647103560, 1652794920, 1653500640, 1653757980, 1653836700, 
          1653812820, 1653748560, 1653835020, 1653751200, 1637182260, 
          1653841320, 1653732000, 1653843120, 1653851700, 1653841380]

# calculate metrics
metrics = []
for index, row in enumerate(trending_data):
    # num stars
    metric = math.log(int(row['num_stars']))
    # num forks scaled by num contributors
    metric += math.log(max(1, float(int(row['num_forks']))))
    # num closed issues minus open issues - more closed issues better (scaled by num contributors)
    diff = math.log(max(1, int(row['num_closed_issues']) - int(row['num_open_issues'])))
    metric += float((int(row['num_closed_issues']) - int(row['num_open_issues']))/max(1, float(row['num_contributors'])))
    time_since = (time.time() - epochs[index])/float(360*24)
    # metric = metric**((0.3)**(-math.log(time_since, 15))) #15
    try:
        metric **= (time_since)**(-math.log(0.5))
    except OverflowError:
        metric = sys.float_info.max
    metric = math.log(metric)
    print("rank:", index+1, " metric:", metric)
    metrics.append(metric)

print("Spearman Rank Correlation: ", rank_correlation(metrics))

rank: 1  metric: 8.325629154799136
rank: 2  metric: 2.0704587570379838
rank: 3  metric: 6.257008307082241
rank: 4  metric: 4.798803780323246
rank: 5  metric: 159.76319492037868
rank: 6  metric: 321.2350382967692
rank: 7  metric: 93.72529743297986
rank: 8  metric: 44.43554100357774
rank: 9  metric: 17.22611216187496
rank: 10  metric: 5.413832781053524
rank: 11  metric: 15.026208221986348
rank: 12  metric: 14.821202872522337
rank: 13  metric: 5.958126307440837
rank: 14  metric: 13.984968183497584
rank: 15  metric: 569.7267113037536
rank: 16  metric: 4.837833731272726
rank: 17  metric: 22.786250200153216
rank: 18  metric: 5.9820406996026785
rank: 19  metric: 3.223058014252584
rank: 20  metric: 4.049853868995256
Spearman Rank Correlation:  0.15639097744360897


In [ ]:
    # factor in author data
    # author_data = ast.literal_eval(row['authors_info'])
    # total_num_contr = 0
    # for author in author_data:
    #     total_num_contr += float(author['metric']['num_of_contri_for_cur_repo'])
    # for author in author_data:
    #     metric += (float(author['metric']['num_of_contri_for_cur_repo'])*float(author['metric']['num_of_followers']))/total_num_contr
    # factor in time since latest commit